In [1]:
import numpy as np
import pandas as pd

import missingno
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import re
from ftfy import fix_text
from num2words import num2words

import textstat

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

import sklearn

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
df_fakejobs = pd.read_csv('../Scraping/Fake Data/cf_fakejobs.csv')
df_cleaned_nonfraud = pd.read_csv('../Scraping/CareersFuture/cleaned_data_nonfraud.csv')

df_fakejobs['fraudulent'] = 1
df_cleaned_nonfraud['fraudulent'] = 0

dataset = pd.concat([df_fakejobs, df_cleaned_nonfraud], ignore_index=True)

dataset.head()

#dataset.to_csv('merged_data.csv', index=False)

,job_post_id,title,address,department,salary_range,company_profile,description,requirement,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,industry,salary_type,fraudulent
0,MCF-2023-03021,Senior Project Manager,"50 Raffles Place, #15-01, Singapore Land Tower...",Project Management,12000-15000,Global Solutions Ltd. is a leading project man...,As a Senior Project Manager at Global Solution...,10+ years of project management experience,Competitive salary and bonus,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Project Management,Monthly,1
1,MCF-2023-03022,Legal Counsel,"6 Battery Road, #20-01, Singapore 049909",Legal,15000-18000,LegalEase Partners is a reputable law firm pro...,LegalEase Partners is looking for a Legal Coun...,Law degree and bar admission,Healthcare and dental coverage,0,1,1,Full-time,Director,4,LegalEase Partners,Legal Services,Monthly,1
2,MCF-2023-03023,Data Analyst (Contract),"3 Church Street, Samsung Hub, Singapore 049483",Data Analysis,6000-8000,TechData Analytics is a data-driven company se...,Join TechData Analytics as a Data Analyst (Con...,Experience in data analysis preferred,Contract-based role,0,1,1,Contract,Senior Level,5,TechData Analytics,Data Analytics,Monthly,1
3,MCF-2023-03024,Marketing Director,"1 Raffles Quay, North Tower, Singapore 048583",Marketing,18000-22000,MaxiMedia Group is a marketing agency speciali...,MaxiMedia Group is in search of a Marketing Di...,10+ years of marketing experience,Performance-based bonuses,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Marketing,Monthly,1
4,MCF-2023-03025,UX/UI Designer,"9 Temasek Boulevard, #04-02, Suntec Tower Two,...",Design,8000-10000,DesignMasters Co. is a design studio dedicated...,DesignMasters Co. is hiring a UX/UI Designer t...,Strong portfolio of design work,Flexible work hours,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Design,Monthly,1


In [37]:
#Country

dataset['country'] = ''

dataset.loc[dataset['address'].notna(), 'country'] = 'SG'

dataset.head()

,job_post_id,title,address,department,salary_range,company_profile,description,requirement,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,industry,salary_type,fraudulent,country
0,MCF-2023-03021,Senior Project Manager,"50 Raffles Place, #15-01, Singapore Land Tower...",Project Management,12000-15000,Global Solutions Ltd. is a leading project man...,As a Senior Project Manager at Global Solution...,10+ years of project management experience,Competitive salary and bonus,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Project Management,Monthly,1,SG
1,MCF-2023-03022,Legal Counsel,"6 Battery Road, #20-01, Singapore 049909",Legal,15000-18000,LegalEase Partners is a reputable law firm pro...,LegalEase Partners is looking for a Legal Coun...,Law degree and bar admission,Healthcare and dental coverage,0,1,1,Full-time,Director,4,LegalEase Partners,Legal Services,Monthly,1,SG
2,MCF-2023-03023,Data Analyst (Contract),"3 Church Street, Samsung Hub, Singapore 049483",Data Analysis,6000-8000,TechData Analytics is a data-driven company se...,Join TechData Analytics as a Data Analyst (Con...,Experience in data analysis preferred,Contract-based role,0,1,1,Contract,Senior Level,5,TechData Analytics,Data Analytics,Monthly,1,SG
3,MCF-2023-03024,Marketing Director,"1 Raffles Quay, North Tower, Singapore 048583",Marketing,18000-22000,MaxiMedia Group is a marketing agency speciali...,MaxiMedia Group is in search of a Marketing Di...,10+ years of marketing experience,Performance-based bonuses,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Marketing,Monthly,1,SG
4,MCF-2023-03025,UX/UI Designer,"9 Temasek Boulevard, #04-02, Suntec Tower Two,...",Design,8000-10000,DesignMasters Co. is a design studio dedicated...,DesignMasters Co. is hiring a UX/UI Designer t...,Strong portfolio of design work,Flexible work hours,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Design,Monthly,1,SG


In [38]:
# salary_group: bin then fill in with most common
# salary_range: fill in with 'unspecified' and use frequency encoding

dataset['salary_group'] = dataset['salary_range']
for idx in (dataset['salary_group'].dropna()).index:
    Range = dataset.at[idx ,'salary_group'].split('-')
    try :
        starts = int(Range[0])
        if len(Range)==2:
          ends = int(Range[1])
        else:
          ends = starts
        salary = (starts + ends)/2
        if salary <= 1000:
          dataset.at[idx ,'salary_group'] = 'extremely low'
        elif salary > 1000 and salary <= 30000:
          dataset.at[idx ,'salary_group'] = 'low'
        elif salary > 30000 and salary <= 100000:
          dataset.at[idx ,'salary_group'] = 'slightly low'
        elif salary > 100000 and salary <= 500000:
          dataset.at[idx ,'salary_group'] = 'slightly high'
        elif salary > 500000 and salary <= 1000000:
          dataset.at[idx ,'salary_group'] = 'high'
        elif salary > 1000000:
          dataset.at[idx ,'salary_group'] = 'extremely high'
    except ValueError:
        dataset.at[idx ,'salary_group'] = np.NaN

dataset.head()

,job_post_id,title,address,department,salary_range,company_profile,description,requirement,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,industry,salary_type,fraudulent,country,salary_group
0,MCF-2023-03021,Senior Project Manager,"50 Raffles Place, #15-01, Singapore Land Tower...",Project Management,12000-15000,Global Solutions Ltd. is a leading project man...,As a Senior Project Manager at Global Solution...,10+ years of project management experience,Competitive salary and bonus,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Project Management,Monthly,1,SG,low
1,MCF-2023-03022,Legal Counsel,"6 Battery Road, #20-01, Singapore 049909",Legal,15000-18000,LegalEase Partners is a reputable law firm pro...,LegalEase Partners is looking for a Legal Coun...,Law degree and bar admission,Healthcare and dental coverage,0,1,1,Full-time,Director,4,LegalEase Partners,Legal Services,Monthly,1,SG,low
2,MCF-2023-03023,Data Analyst (Contract),"3 Church Street, Samsung Hub, Singapore 049483",Data Analysis,6000-8000,TechData Analytics is a data-driven company se...,Join TechData Analytics as a Data Analyst (Con...,Experience in data analysis preferred,Contract-based role,0,1,1,Contract,Senior Level,5,TechData Analytics,Data Analytics,Monthly,1,SG,low
3,MCF-2023-03024,Marketing Director,"1 Raffles Quay, North Tower, Singapore 048583",Marketing,18000-22000,MaxiMedia Group is a marketing agency speciali...,MaxiMedia Group is in search of a Marketing Di...,10+ years of marketing experience,Performance-based bonuses,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Marketing,Monthly,1,SG,low
4,MCF-2023-03025,UX/UI Designer,"9 Temasek Boulevard, #04-02, Suntec Tower Two,...",Design,8000-10000,DesignMasters Co. is a design studio dedicated...,DesignMasters Co. is hiring a UX/UI Designer t...,Strong portfolio of design work,Flexible work hours,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Design,Monthly,1,SG,low


In [39]:
#Text

dataset['title_contain_urgent'] = dataset['title'].apply(lambda x: 1 if 'urgent' in x.lower() else 0)
dataset['have_company_profile'] = np.where(dataset['company_profile'].notnull(), 1, 0)
dataset['have_description'] = np.where(dataset['description'].notnull(), 1, 0)
dataset['have_requirements'] = np.where(dataset['requirement'].notnull(), 1, 0)
dataset['have_benefits'] = np.where(dataset['benefits'].notnull(), 1, 0)

dataset[['company_profile']] = dataset[['company_profile']].fillna('')
dataset[['description']] = dataset[['description']].fillna('')
dataset[['requirement']] = dataset[['requirement']].fillna('')
dataset[['benefits']] = dataset[['benefits']].fillna('')
dataset['full_text'] = dataset['title'] + ' ' + dataset['company_profile'] + ' ' + dataset['description'] + ' ' + dataset['requirement'] + ' ' + dataset['benefits']

dataset.head()

,job_post_id,title,address,department,salary_range,company_profile,description,requirement,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,industry,salary_type,fraudulent,country,salary_group,title_contain_urgent,have_company_profile,have_description,have_requirements,have_benefits,full_text
0,MCF-2023-03021,Senior Project Manager,"50 Raffles Place, #15-01, Singapore Land Tower...",Project Management,12000-15000,Global Solutions Ltd. is a leading project man...,As a Senior Project Manager at Global Solution...,10+ years of project management experience,Competitive salary and bonus,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Project Management,Monthly,1,SG,low,0,1,1,1,1,Senior Project Manager Global Solutions Ltd. i...
1,MCF-2023-03022,Legal Counsel,"6 Battery Road, #20-01, Singapore 049909",Legal,15000-18000,LegalEase Partners is a reputable law firm pro...,LegalEase Partners is looking for a Legal Coun...,Law degree and bar admission,Healthcare and dental coverage,0,1,1,Full-time,Director,4,LegalEase Partners,Legal Services,Monthly,1,SG,low,0,1,1,1,1,Legal Counsel LegalEase Partners is a reputabl...
2,MCF-2023-03023,Data Analyst (Contract),"3 Church Street, Samsung Hub, Singapore 049483",Data Analysis,6000-8000,TechData Analytics is a data-driven company se...,Join TechData Analytics as a Data Analyst (Con...,Experience in data analysis preferred,Contract-based role,0,1,1,Contract,Senior Level,5,TechData Analytics,Data Analytics,Monthly,1,SG,low,0,1,1,1,1,Data Analyst (Contract) TechData Analytics is ...
3,MCF-2023-03024,Marketing Director,"1 Raffles Quay, North Tower, Singapore 048583",Marketing,18000-22000,MaxiMedia Group is a marketing agency speciali...,MaxiMedia Group is in search of a Marketing Di...,10+ years of marketing experience,Performance-based bonuses,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Marketing,Monthly,1,SG,low,0,1,1,1,1,Marketing Director MaxiMedia Group is a market...
4,MCF-2023-03025,UX/UI Designer,"9 Temasek Boulevard, #04-02, Suntec Tower Two,...",Design,8000-10000,DesignMasters Co. is a design studio dedicated...,DesignMasters Co. is hiring a UX/UI Designer t...,Strong portfolio of design work,Flexible work hours,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Design,Monthly,1,SG,low,0,1,1,1,1,UX/UI Designer DesignMasters Co. is a design s...


In [40]:
# drop useless columns
drop_cols = ['job_post_id','address','company_profile','description','requirement','benefits', 'industry']

dataset = dataset.drop(drop_cols,axis=1)

# fill in with the help of key words from 'title'
str_list = ['Engineer','Account','Accountant','Accounting','Developer','Sales','Marketing','Beauty','Service','Customer','Consumer',
'Customer','Writer','Pet','Architect', 'Dog', 'Game', 'Mobile', 'Design', 'Financial','Programmer']

for strs in str_list:
    dataset.loc[(dataset['title'].str.contains(strs, case=False, na=False)) & (dataset['department'].isna()),'department'] = \
    dataset[dataset['title'].str.contains(strs, case=False, na=False)]['department'].mode()[0]

dataset['department'] = dataset['department'].fillna('unspecified')

#rename department to category
dataset.rename(columns={"department": "category"}, inplace=True)

dataset.head()

,title,category,salary_range,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,salary_type,fraudulent,country,salary_group,title_contain_urgent,have_company_profile,have_description,have_requirements,have_benefits,full_text
0,Senior Project Manager,Project Management,12000-15000,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Monthly,1,SG,low,0,1,1,1,1,Senior Project Manager Global Solutions Ltd. i...
1,Legal Counsel,Legal,15000-18000,0,1,1,Full-time,Director,4,LegalEase Partners,Monthly,1,SG,low,0,1,1,1,1,Legal Counsel LegalEase Partners is a reputabl...
2,Data Analyst (Contract),Data Analysis,6000-8000,0,1,1,Contract,Senior Level,5,TechData Analytics,Monthly,1,SG,low,0,1,1,1,1,Data Analyst (Contract) TechData Analytics is ...
3,Marketing Director,Marketing,18000-22000,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Monthly,1,SG,low,0,1,1,1,1,Marketing Director MaxiMedia Group is a market...
4,UX/UI Designer,Design,8000-10000,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Monthly,1,SG,low,0,1,1,1,1,UX/UI Designer DesignMasters Co. is a design s...


In [41]:
#merge already and discard duplicated words
def remove_duplicates(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))
    return " ".join(unique_words)

dataset['category'] = dataset['category'].apply(remove_duplicates)

dataset.head()

,title,category,salary_range,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,salary_type,fraudulent,country,salary_group,title_contain_urgent,have_company_profile,have_description,have_requirements,have_benefits,full_text
0,Senior Project Manager,Project Management,12000-15000,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Monthly,1,SG,low,0,1,1,1,1,Senior Project Manager Global Solutions Ltd. i...
1,Legal Counsel,Legal,15000-18000,0,1,1,Full-time,Director,4,LegalEase Partners,Monthly,1,SG,low,0,1,1,1,1,Legal Counsel LegalEase Partners is a reputabl...
2,Data Analyst (Contract),Data Analysis,6000-8000,0,1,1,Contract,Senior Level,5,TechData Analytics,Monthly,1,SG,low,0,1,1,1,1,Data Analyst (Contract) TechData Analytics is ...
3,Marketing Director,Marketing,18000-22000,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Monthly,1,SG,low,0,1,1,1,1,Marketing Director MaxiMedia Group is a market...
4,UX/UI Designer,Design,8000-10000,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Monthly,1,SG,low,0,1,1,1,1,UX/UI Designer DesignMasters Co. is a design s...


In [42]:
#textstat - check text complexity (python package) https://pypi.org/project/textstat/

import textstat

# Calculate readability scores for the "category" column
dataset['cat_flesch_score'] = dataset['category'].apply(textstat.flesch_reading_ease) #low score = very confusing
dataset['cat_fk_grade_level'] = dataset['category'].apply(textstat.flesch_kincaid_grade)

dataset.head()

,title,category,salary_range,telecommuting,has_company_logo,has_questions,employment_type,required_experience,min_experience,company,salary_type,fraudulent,country,salary_group,title_contain_urgent,have_company_profile,have_description,have_requirements,have_benefits,full_text,cat_flesch_score,cat_fk_grade_level
0,Senior Project Manager,Project Management,12000-15000,0,0,1,Full-time,Mid-Level,8,Global Solutions Ltd.,Monthly,1,SG,low,0,1,1,1,1,Senior Project Manager Global Solutions Ltd. i...,35.61,8.8
1,Legal Counsel,Legal,15000-18000,0,1,1,Full-time,Director,4,LegalEase Partners,Monthly,1,SG,low,0,1,1,1,1,Legal Counsel LegalEase Partners is a reputabl...,36.62,8.4
2,Data Analyst (Contract),Data Analysis,6000-8000,0,1,1,Contract,Senior Level,5,TechData Analytics,Monthly,1,SG,low,0,1,1,1,1,Data Analyst (Contract) TechData Analytics is ...,-6.70,14.7
3,Marketing Director,Marketing,18000-22000,0,0,0,Full-time,Senior Level,2,MaxiMedia Group,Monthly,1,SG,low,0,1,1,1,1,Marketing Director MaxiMedia Group is a market...,-47.99,20.2
4,UX/UI Designer,Design,8000-10000,0,0,1,Full-time,Senior Level,0,DesignMasters Co.,Monthly,1,SG,low,0,1,1,1,1,UX/UI Designer DesignMasters Co. is a design s...,36.62,8.4
